# MDT Operations Playbook for the Necronomicon Laptop

## Initial Setup
1. Import required packages:

In [19]:
import os
import netifaces
print("Imports Done")

Imports Done


2. Update suricata signatures.

In [20]:
os.system("sudo suricata-update")

9/9/2022 -- 15:49:58 - <Info> -- Using data-directory /var/lib/suricata.
9/9/2022 -- 15:49:58 - <Info> -- Using Suricata configuration /etc/suricata/suricata.yaml
9/9/2022 -- 15:49:58 - <Info> -- Using /etc/suricata/rules for Suricata provided rules.
9/9/2022 -- 15:49:58 - <Info> -- Found Suricata version 6.0.6 at /usr/bin/suricata.
9/9/2022 -- 15:49:58 - <Info> -- Loading /etc/suricata/suricata.yaml
9/9/2022 -- 15:49:58 - <Info> -- Disabling rules for protocol mqtt
9/9/2022 -- 15:49:58 - <Info> -- Disabling rules for protocol http2
9/9/2022 -- 15:49:58 - <Info> -- Disabling rules for protocol dnp3
9/9/2022 -- 15:49:58 - <Info> -- Disabling rules for protocol enip
9/9/2022 -- 15:49:58 - <Info> -- Checking https://malsilo.gitlab.io/feeds/dumps/malsilo.rules.tar.gz.md5.
9/9/2022 -- 15:49:58 - <Info> -- Remote checksum has not changed. Not fetching.
9/9/2022 -- 15:49:58 - <Info> -- Fetching https://security.etnetera.cz/feeds/etn_aggressive.rules.
9/9/2022 -- 15:50:06 - <Info> -- Done.
9/9

0

3. Validate Disk Usage

In [8]:
os.system("sudo df -h | awk '{ if ($6==\"/\") print $1,$5,$6 }'")

/dev/sda2 6% /


0

## Elastic Stack Operation

If you want to review prior analysis, or get ready to injest data into the elk stack for analysis, first start the elastic containers.

In [4]:
os.system("sudo docker start es01")
os.system("sudo docker start kibana")
# check

es01
kibana


0

# Collection and Analysis
## Capture or Load PCAP
Whether you are loading a PCAP from another source. Or capturing from this device. The PCAP should be copied into the **/home/mdtoperator/collected_pcaps** folder.

### Collect Live Data

1. Run tcpdump agains the desired interface.

In [18]:
netifaces; if_list = netifaces.interfaces()

print("Ethernet Interface: " + if_list[1])
print("Wifi Interface: " + if_list[2])

Ethernet Interface:enp0s31f6
Wifi Interface: wlp2s0


Note the interface that you want to capture from above and modify the <interface> in the below command to begin capturing.

In [ ]:
os.system("sudo tcpdump -i <interface> -w /home/mdtoperator/collected_pcaps")

## Zeek Analysis
First, run zeek against the new pcaps in the collected_pcaps folder. This will dump the zeek logs in the configured folder.

In [ ]:
os.system("sudo zeek -r /home/mdtoperator/Desktop/collected_pcaps/<pcap_name> LogAscii::use_json=T Log::default_logdir=/usr/local/zeek/logs/current")

## Suricata Analsysis
Now run suricata agains the new pcap in the collected_pcaps folder. This will dump the suricata logs in the configured folder.

In [ ]:
os.system("sudo suricata -c /etc/suricata/suricata.yaml -r /home/mdtoperator/Desktop/collected_pcaps/<pcap_name> -k none -l /var/log/suricata/")

## Filebeat Into Elastic

In [ ]:
os.system("sudo filebeat -e")
os.system("browser http://0.0.0.0:5601")

The browser will open to Kiban for you to browse through your new data.